# Mini Project II 

## Motivation
Which API has a better coverage of the area?
- tough to have a hypothesis here because in my world, neither are used!
- assumption is for FourSquare to be better since it's more about location and business details, YELP is subjective (& why it failed?)

Choose the top 10 POIs based on the popularity (number of reviews or average rating)

## Project Flow 
- What steps does the project have?
1) planning - including project definitions
2) get api results - FourSquare, Yelp
3) Use pandas to make Data Frame
4) Design DB structure
5) Make DB
6) Sort results for FourSquare, Yelp
7) According to definitions, which has better coverage?
8) Takeaways, Learnings, Risks, Opportunities

## Task
- For this project I need to use APIs to pull in data
- Build database of restaurants, bars and other POIs in the area of your choice (Marda Loop)
    - What are my POIs?
        - can use comma separated Ids
        - coffee shops, restaurants, nightlife *opportunity to add more POIs*
FourSquare: CategoryId = 
food = 4d4b7105d754a06374d81259
coffee = 4bf58dd8d48988d1e0931735
night life spot = 4d4b7105d754a06376d81259 (same as food)

Yelp: categories = 
coffee = coffee
Restaurants = restaurants
food = food *includes groceries*
nightlife = nightlife

- For area, can I create a box around Marda Loop like Google Maps does?
    - Crowchild, 33rd AVE SW, 50 AVE SW, 14 St SW
    - Pin1 = 51.023210431410085, -114.11773309036127
    - Pin2 = 51.02317901782409, -114.0948098715688
    - Pin3 = 51.0086636643716, -114.08572049069683
    - Pin4 = 51.00875793457025, -114.117982798627
    - consider instead using "marda loop" as single location instead of lat/longs to reduce calls 
    - marda loop no geocode, so use sing lat/long and use 1km radius

FourSquare: ll = 51.016007, -114.104438, radius = 1000
Yelp:

- Once I have the data I need to create a database to store the data
    - Lucid Chart

## Approach
- How do I define better coverage? 
    - No of businesses found/area
    - Decided to go with a small area because I know it has a large concentration of businesses,
        - wanted to conserve my API calls (didn't work!)
        - isolated query to 'food and drink'
- Rank Venues by Average Rating, .head(10)
    - FourSquare: use Get Venue Recommendations URL, params sortByPopularity=1
    - Yelp: use param sort_by=rating 

## Results
What are the takeaways here? Big picture!
So what? What does this mean for a Data Scientist?
- The available data for the same questions provided some overlap, some gaps filled, some techincal issues, some bad data, some missing data
- Therefore utilizing multiple, perhaps similar data sources for a project. Assessing data quality and the impacts that could have on your results
- Eg. getting bad quality data from FourSquare but it is known (in Canada at least) that Yelp is a poor database due to heavily skewed information
- However, if you know anything about reviews/ratings, you shouldn't trust it as a metric generally speaking, so it's not necessarily a great assessment of quality of API source. (But I digress) If I had more time, I would do a broader sweep of the data in each API.

## Conclusion
- Really wanted to challenge myself to think Big Picture. Very easy to get lost in the coding
- Utilize auxilary skill sets - project management
- Which API was my favourite? Yelp
- What was biggest challenge? Parsing the JSON outputs. Really wish there was a more straightforward simplistic (eg software black box solution) way to get data. I can see this being a perpetual problem that I best start to get comfortable around... we know that there is so much more data being created than ever before and it is a mess. There is no one solution. There's just too much of it. That's why the world needs Data Scientists. To help clean it up, make sense of it, put it to use. To do something with it. 


## Set up the Jupyter!

In [1]:
import requests
import json
import pandas as pd
from IPython.display import JSON

## Get the APIs running

In [2]:
def get_api(url):
    # We send the request to the API
    res = requests.get(url)
    # We can check if the request was successful
    #print(res.status_code)
    return res.json()

In [3]:
url_FS = 'https://api.foursquare.com/v2/venues/explore?client_id=UWJ5FNNWUM2ID3KT3II3QD55XXHSF0FEXLINGPV3K5LQZSQQ&client_secret=LWBMU4Z3ANKEWKEHUIE2TQABQV234S1WB3K2JYAHV2LNXVYL&v=20201231&ll=51.016007, -114.104438&radius=1000&sortByPopularity=1&categoryId=4d4b7105d754a06374d81259,4bf58dd8d48988d1e0931735'
FS = get_api(url_FS)
JSON(FS)

<IPython.core.display.JSON object>

In [4]:
dfFS = pd.DataFrame(FS['response']['groups'][0]['items'])
pd.json_normalize(FS['response']['groups'][0]['items']).drop(columns='referralId').head(0)
#remove additional columns

,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.crossStreet


In [5]:
FS_rows=[]

for row in FS['response']['groups'][0]['items']:
    venue = row['venue']
    venue_name = venue['name']
    category = venue['categories'][0]
    category_name = category['name']
    venue_id = venue['id']
    #add more if needed
    url_rating = f'https://api.foursquare.com/v2/venues/{venue_id}?client_id=UWJ5FNNWUM2ID3KT3II3QD55XXHSF0FEXLINGPV3K5LQZSQQ&client_secret=LWBMU4Z3ANKEWKEHUIE2TQABQV234S1WB3K2JYAHV2LNXVYL&v=20201231'
    rating = get_api(url_rating)
    if 'rating' in rating['response']['venue']:
        rating = rating['response']['venue']['rating']
    else:
        rating = None
    FS_rows.append({'venue':venue_name,'category_name':category_name, 'venue_id':venue_id, 'rating':rating})

In [6]:
dfFS = pd.DataFrame(FS_rows).sort_values('rating', ascending=False).head(10)
dfFS['Source'] = 'FS'
dfFS

,venue,category_name,venue_id,rating,Source
0,Original Joe's Restaurant & Bar,Restaurant,4b0586e8f964a520567422e3,7.7,FS
8,Avenue Deli,Sandwich Place,4b60f88bf964a52011042ae3,7.7,FS
5,nho saigon,Vietnamese Restaurant,4b735e50f964a5200dab2de3,7.5,FS
2,Globefish,Sushi Restaurant,4bdcc1dcc79cc9287a0388e9,7.4,FS
3,Belmont Diner,Breakfast Spot,540b2114498e7fb901fb131a,7.4,FS
7,Subway,Sandwich Place,4c0d5650336220a19192ca77,6.3,FS
1,Boogies Burgers,Burger Joint,596bb0569cadd94f2c5a975e,NaN,FS
4,Pegasus Restaurant,Greek Restaurant,4e2192a962e1964dbb6813db,NaN,FS
6,Dragon Gate Restaurant,Chinese Restaurant,4dd46b20ae60a78cec56f67c,NaN,FS
9,Globefish Sushi & Izakaya,Sushi Restaurant,4b630408f964a520ca5d2ae3,NaN,FS


In [7]:
Yelp = 'C:/Users/stace/data_bootcamp/w2/d4-day_4/mini-project-II/Yelp_response.json'
JSON(Yelp)
with open(Yelp, 'r') as f:
    text = f.read()
Yelp_json = json.loads(text)

In [8]:
Yelp_rows=[]

for row in Yelp_json['businesses']:
    venue_name = row['name']
    category = row['categories'][0]
    category_name = category['title']
    ratings = row['rating']
    Yelp_rows.append({'venue':venue_name,'category_name':category_name, 'rating':ratings}) #add rating

dfYelp = pd.DataFrame(Yelp_rows).sort_values('rating', ascending=False)
dfYelp['Source'] = 'Yelp'
dfYelp

,venue,category_name,rating,Source
4,Pacific Poke,Poke,5.0,Yelp
0,Monogram Coffee,Coffee & Tea,4.5,Yelp
2,Village Ice Cream,Ice Cream & Frozen Yogurt,4.5,Yelp
18,OPA! of Greece Marda Loop,Greek,4.5,Yelp
9,Master Chocolat,Chocolatiers & Shops,4.5,Yelp
3,Belmont Diner,Diners,4.0,Yelp
5,Boogie's Burgers,Burgers,4.0,Yelp
7,El's Japanese Fusion,Sushi Bars,4.0,Yelp
8,Globefish Sushi & Izakaya,Sushi Bars,4.0,Yelp
1,Phil & Sebastian Coffee Roasters,Coffee & Tea,4.0,Yelp


## Connect to SQLite3

In [9]:
import sqlite3
from sqlite3 import Error

def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection
connection1 = create_connection("Project.db")
cursor = connection1.cursor()

Connection to SQLite DB successful


In [10]:
connection1

In [11]:
cursor.execute("DROP TABLE FS_SQL")
dfFS.to_sql('FS_SQL', con=connection1)
connection1.execute('SELECT * FROM FS_SQL').fetchall()

[(0,
  "Original Joe's Restaurant & Bar",
  'Restaurant',
  '4b0586e8f964a520567422e3',
  7.7,
  'FS'),
 (8, 'Avenue Deli', 'Sandwich Place', '4b60f88bf964a52011042ae3', 7.7, 'FS'),
 (5,
  'nho saigon',
  'Vietnamese Restaurant',
  '4b735e50f964a5200dab2de3',
  7.5,
  'FS'),
 (2, 'Globefish', 'Sushi Restaurant', '4bdcc1dcc79cc9287a0388e9', 7.4, 'FS'),
 (3, 'Belmont Diner', 'Breakfast Spot', '540b2114498e7fb901fb131a', 7.4, 'FS'),
 (7, 'Subway', 'Sandwich Place', '4c0d5650336220a19192ca77', 6.3, 'FS'),
 (1,
  'Boogies Burgers',
  'Burger Joint',
  '596bb0569cadd94f2c5a975e',
  None,
  'FS'),
 (4,
  'Pegasus Restaurant',
  'Greek Restaurant',
  '4e2192a962e1964dbb6813db',
  None,
  'FS'),
 (6,
  'Dragon Gate Restaurant',
  'Chinese Restaurant',
  '4dd46b20ae60a78cec56f67c',
  None,
  'FS'),
 (9,
  'Globefish Sushi & Izakaya',
  'Sushi Restaurant',
  '4b630408f964a520ca5d2ae3',
  None,
  'FS')]

In [12]:
cursor.execute("DROP TABLE Yelp_SQL")
dfYelp.to_sql('Yelp_SQL', con=connection1)
connection1.execute('SELECT * FROM Yelp_SQL').fetchall()

[(4, 'Pacific Poke', 'Poke', 5.0, 'Yelp'),
 (0, 'Monogram Coffee', 'Coffee & Tea', 4.5, 'Yelp'),
 (2, 'Village Ice Cream', 'Ice Cream & Frozen Yogurt', 4.5, 'Yelp'),
 (18, 'OPA! of Greece Marda Loop', 'Greek', 4.5, 'Yelp'),
 (9, 'Master Chocolat', 'Chocolatiers & Shops', 4.5, 'Yelp'),
 (3, 'Belmont Diner', 'Diners', 4.0, 'Yelp'),
 (5, "Boogie's Burgers", 'Burgers', 4.0, 'Yelp'),
 (7, "El's Japanese Fusion", 'Sushi Bars', 4.0, 'Yelp'),
 (8, 'Globefish Sushi & Izakaya', 'Sushi Bars', 4.0, 'Yelp'),
 (1, 'Phil & Sebastian Coffee Roasters', 'Coffee & Tea', 4.0, 'Yelp'),
 (11, 'K-Thi Viet Cuisine', 'Vietnamese', 4.0, 'Yelp'),
 (12, 'Pegasus', 'Greek', 4.0, 'Yelp'),
 (13, 'Tres Marias Mexican Market', 'International Grocery', 4.0, 'Yelp'),
 (14, 'The Garrison Pub', 'Pubs', 3.5, 'Yelp'),
 (10, 'Nho Saigon', 'Vietnamese', 3.5, 'Yelp'),
 (6, 'My Favorite Ice Cream Shoppe', 'Ice Cream & Frozen Yogurt', 3.5, 'Yelp'),
 (19, "Bell's Bookstore Cafe", 'Coffee & Tea', 3.5, 'Yelp'),
 (15, 'The Trop Bar 